In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from IPython.display import clear_output
import re

In [3]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')

SessionNotCreatedException: Message: session not created
from chrome not reachable
  (Session info: chrome=75.0.3770.100)


### 1. Definujemy funkcje potrzebne do pobrania danych

In [76]:

def next_page():
    element = wd.find_elements_by_class_name('pagination-next')
    element[0].click()
    
def if_next_page():
    element = wd.find_elements_by_class_name('pagination-next')
    next_page_exists=True
    if element[0].get_attribute('href') == 'https://www.hltv.org/stats/matches?offset=0':
        next_page_exists=False
    return next_page_exists
    
    
#     return element[0].get_attribute('href')
    
def previous_page():
    element = wd.find_elements_by_class_name('pagination-prev')
    element[0].click()


def go_back_page():
    wd.back()

    
###dane_tabel    
def get_info_from_main_tabels():
    results_table = []
    tables = wd.find_elements_by_class_name('stats-table') 
    
    for table in tables:
        rows = table.find_elements_by_tag_name('tr')
        columns_names=get_columns_names(table)
    
    for row in rows:
        columns = row.find_elements_by_css_selector("td")
        temp = []
        for column in columns:
            temp.append(column.text)
        temp.append(get_link_summary(row))
        results_table.append(temp)
        df_out = pd.DataFrame(results_table)
        
        
    return df_out, columns_names



###nazwy kolumn
def get_columns_names(table):
    columns_names = []
    table_head = table.find_elements_by_tag_name('th')
    for head in table_head:
        columns_names.append(head.text)
    return columns_names
    
    
###pobierz link z glownej strony   
def get_link_summary(row):
    columns_link = row.find_elements_by_class_name('date-col')
    link=''
    for column in columns_link:
        if column.find_elements_by_css_selector('a'):
            temp2 = column.find_elements_by_css_selector('a')[0]
            link = temp2.get_attribute('href')
    return(link)
    
    
def in_database(df, last_element):
    m =  df.isin([last_element]).any()
    m = m.any()
    return m
 
    
## nowe dane do tabeli glownej

def get_new_data_to_main_table():
    df=pd.DataFrame()
    df_new, _ =  get_info_from_main_tabels()
    df = pd.concat([df, df_new])
    last_value=df.iloc[-1][5]

    while ~in_database(df_main_table, last_value):
        next_page()
        df_new, _ =  get_info_from_main_tabels()
        df = pd.concat([df, df_new])
        last_value = df.iloc[-1][5]
    df = process_main_table(df)
    df = df[~df['link'].isin(df_main_table['link'])]
    return df
    
    

    
    
# funkcje do czyszczenia tabeli glownej


#oczysczanie glownej tabeli
def process_main_table(df):
    df = df.dropna()
    columns_df_main = ['date', 'team1', 'team2', 'map', 'event', 'link']
    df.columns = columns_df_main
    df.reset_index(drop=True, inplace=True)
    df['score_team1'] = df.apply(lambda x: get_score_from_main_table(x['team1']), axis=1)
    df['team1'] = df.apply(lambda x: get_name_from_main_table(x['team1']), axis=1)
    df['score_team2'] = df.apply(lambda x: get_score_from_main_table(x['team2']), axis=1)
    df['team2'] = df.apply(lambda x: get_name_from_main_table(x['team2']), axis=1)
    df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%y')
    return df

# wyczysc z nazwy wynik
def get_name_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        row_text = row_text.replace(" "+m[0],'')
    return row_text    



# wez z nazwy wynik
def get_score_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        score = m[1]
    else:
        score = np.nan
    return score

    
## FUNKCJE DO TABEL ZE SZCZEGOLAMI

def get_info_from_stats():
    list_info = {}
    list1 = wd.find_element_by_class_name('stats-match-maps') 
    list2 = list1.find_elements_by_class_name('stats-match-map')
    if list2:
        list_info['score_total'] = list2[0].text.split('\n')[0]
        list_info['match_type'] = list2[0].text.split('\n')[1]
        for idx, li in enumerate(list2):
            if(li.get_attribute("class") == 'col stats-match-map standard-box a-reset'):
                list_info['map_number'] = idx
    else:
        list_info['score_total'] = np.nan
        list_info['match_type'] = 'Best of 1'
        list_info['map_number'] = 1    
    return(list_info)

def get_match_info():
    dict_match_info={}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_element_by_class_name('match-info-box')
    list_match_info = match_info.text.split("\n")
    dict_match_info['event'] = list_match_info[0]
    dict_match_info['date_time'] = list_match_info[1]
    dict_match_info['map_name'] = list_match_info[4]
    dict_match_info['score_team1'] = list_match_info[6]
    dict_match_info['score_team2'] = list_match_info[8]
    dict_match_info['team1'] = list_match_info[5]
    dict_match_info['team2'] = list_match_info[7]
    return(dict_match_info)

def get_match_info_details():
    dict_match_info_more = {}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_elements_by_class_name('match-info-row')
    list_scores = match_info[0].find_elements_by_css_selector('span')
    dict_match_info_more['team1side'] = list_scores[2].get_attribute("class")
    dict_match_info_more['team1_score_1half']= list_scores[2].text
    dict_match_info_more['team2_score_1half']= list_scores[3].text
    dict_match_info_more['team1_score_2half']= list_scores[4].text
    dict_match_info_more['team2_score_2half']= list_scores[5].text
    dict_match_info_more['team1_rating'] = match_info[1].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_rating'] = match_info[1].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[2]
    return(dict_match_info_more)

def get_info_players():
    dict_players={}
    tables =wd.find_elements_by_class_name('stats-table') 
    for index_table,table in enumerate(tables):
        rows = table.find_elements_by_tag_name('tr')
        for index, row in enumerate(rows):
            temp = []
            columns = row.find_elements_by_css_selector("td")
            for column in columns:
                temp.append(column.text)
            if temp:
                dict_players.update(players_dict_create(temp, index, index_table+1))
    return (dict_players)

def players_dict_create(row, row_number, team_number):
    dict_to_return={}
    list_of_keys=['player_name','p_kills','p_asists','p_death','p_kast','p_ADR','p_rating']
    player_number = row_number
    dict_to_return[f'team{team_number}_{list_of_keys[0]}_{player_number}']=row[0]
    dict_to_return[f'team{team_number}_{list_of_keys[1]}_{player_number}']=row[1]
    dict_to_return[f'team{team_number}_{list_of_keys[2]}_{player_number}']=row[2]
    dict_to_return[f'team{team_number}_{list_of_keys[3]}_{player_number}']=row[3]
    dict_to_return[f'team{team_number}_{list_of_keys[4]}_{player_number}']=row[4]
    dict_to_return[f'team{team_number}_{list_of_keys[5]}_{player_number}']=row[6]
    dict_to_return[f'team{team_number}_{list_of_keys[6]}_{player_number}']=row[8]
    return dict_to_return
        



    
    



### 2.Pobieramy dane z całosci - tylko gdy musimy odtworzyc cala baze

#### 2.1 pobieramy dane z glownej strony

In [56]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')

In [52]:

#pobieramy dane z glownej
df_all=pd.DataFrame()
while if_next_page():
    df, columns = get_info_from_main_tabels()
    next_page()
    df_all = pd.concat([df_all, df])
    print([wd.find_element_by_class_name('pagination-data').text])
    clear_output(wait=True)

TypeError: 'NoneType' object is not subscriptable

In [54]:
#oczyszczamy dane
df_main = process_main_table(df_all)
df_main.to_pickle('main_table_all_data.pkl')





C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

,date,team1,team2,map,event,link,score_team1,score_team2
0,2019-07-10,DreamEaters,Vega Squadron,Overpass,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
1,2019-07-10,Vega Squadron,DreamEaters,Dust2,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
2,2019-07-10,eNergy,Goliath,Vertigo,ESEA Open Season 31 South Africa,https://www.hltv.org/stats/matches/mapstatsid/...,25,28
3,2019-07-10,PACT,Chaos,Train,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,12,16
4,2019-07-10,PACT,Chaos,Overpass,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
5,2019-07-10,HAVU,Ancient,Nuke,GameAgents League Season 3,https://www.hltv.org/stats/matches/mapstatsid/...,16,11
6,2019-07-10,HAVU,Ancient,Dust2,GameAgents League Season 3,https://www.hltv.org/stats/matches/mapstatsid/...,16,4
7,2019-07-10,NoChance,Aristocracy,Train,UCC Summer Smash,https://www.hltv.org/stats/matches/mapstatsid/...,9,16
8,2019-07-10,Aristocracy,NoChance,Nuke,UCC Summer Smash,https://www.hltv.org/stats/matches/mapstatsid/...,16,6
9,2019-07-10,Aristocracy,NoChance,Dust2,UCC Summer Smash,https://www.hltv.org/stats/matches/mapstatsid/...,12,16



#### 2.2 pobieramy dane szczegolowe


In [57]:
df = pd.DataFrame()
number_of_rows = len(df_main.index)

for index, row in df_main.iterrows():
    try:
        dict_temp={}
        link = row['link']
        wd.get(link)
        dict_temp = get_info_from_stats()
        dict_temp.update(get_match_info())
        dict_temp.update(get_match_info_details())
        dict_temp.update(get_info_players())
        dict_temp.update({'link' : link})
        df = df.append(dict_temp,ignore_index=True)
        print(f'{index}/{number_of_rows}')
        clear_output(wait=True)
    except NoSuchElementException:
        pass
    if (index%1000) == 0 :
        df.to_pickle('details_all.pkl')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.100)


#### 2.3 laczymy wczesniej pobrane bazy

In [58]:
df_main_table = pd.read_pickle('main_table.pkl')
df_details = pd.read_pickle('details.pkl')

In [59]:
df_data = pd.merge(df_main_table, df_details, on=['link'], how='left', suffixes=['main', 'details'] )

In [62]:
df_data.to_pickle('df_data.pkl')

### 3.Pobieramy nowe dane 

#### 3.1 pobieramy z glownej tabeli

In [64]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')
df_main_table = pd.read_pickle('main_table.pkl')

df=pd.DataFrame()
df_new, _ =  get_info_from_main_tabels()
df = pd.concat([df, df_new])
last_value=df.iloc[-1][5]

while ~in_database(df_main_table, last_value):
    next_page()
    df_new, _ =  get_info_from_main_tabels()
    df = pd.concat([df, df_new])
    last_value = df.iloc[-1][5]

In [68]:
df_main = process_main_table(df)

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [69]:
#wycinamy te ktore juz sa w bazie
df_main = df_main[~df_main['link'].isin(df_main_table['link'])]

In [70]:
df_main.to_pickle('main_table_new_data.pkl')

#### 3.2 pobieramy dane szczegolowe

In [71]:
df = pd.DataFrame()
number_of_rows = len(df_main.index)

for index, row in df_main.iterrows():
    try:
        dict_temp={}
        link = row['link']
        wd.get(link)
        dict_temp = get_info_from_stats()
        dict_temp.update(get_match_info())
        dict_temp.update(get_match_info_details())
        dict_temp.update(get_info_players())
        dict_temp.update({'link' : link})
        df = df.append(dict_temp,ignore_index=True)
        print(f'{index}/{number_of_rows}')
        clear_output(wait=True)
    except NoSuchElementException:
        pass
    if (index%1000) == 0 :
        df.to_pickle('details_new.pkl')

853/854


In [72]:
df.to_pickle('details_new.pkl')

#### 3.3 laczymy wczesniej pobrane bazy


In [73]:
df_main_table = pd.read_pickle('main_table_new_data.pkl')
df_details = pd.read_pickle('details_new.pkl')

In [74]:
df_data = pd.merge(df_main_table, df_details, on=['link'], how='left', suffixes=['main', 'details'] )

In [75]:
df_data.to_pickle('df_data_new.pkl')

In [69]:
df_data = pd.read_pickle('df_data.pkl')
df_data_new = pd.read_pickle('df_data_new.pkl')

In [49]:
lsit = df_data_new.columns
for p in lsit: print( p)

date
team1main
team2main
map
eventmain
link
score_team1main
score_team2main
date_time
eventdetails
map_name
map_number
match_type
score_team1details
score_team2details
score_total
team1details
team1_clutches_won
team1_first_kills
team1_p_ADR_1
team1_p_ADR_2
team1_p_ADR_3
team1_p_ADR_4
team1_p_ADR_5
team1_p_asists_1
team1_p_asists_2
team1_p_asists_3
team1_p_asists_4
team1_p_asists_5
team1_p_death_1
team1_p_death_2
team1_p_death_3
team1_p_death_4
team1_p_death_5
team1_p_kast_1
team1_p_kast_2
team1_p_kast_3
team1_p_kast_4
team1_p_kast_5
team1_p_kills_1
team1_p_kills_2
team1_p_kills_3
team1_p_kills_4
team1_p_kills_5
team1_p_rating_1
team1_p_rating_2
team1_p_rating_3
team1_p_rating_4
team1_p_rating_5
team1_player_name_1
team1_player_name_2
team1_player_name_3
team1_player_name_4
team1_player_name_5
team1_rating
team1_score_1half
team1_score_2half
team1side
team2details
team2_clutches_won
team2_first_kills
team2_p_ADR_1
team2_p_ADR_2
team2_p_ADR_3
team2_p_ADR_4
team2_p_ADR_5
team2_p_asists_1

In [70]:
df_data_new = df_data_new.drop(columns=['team1_p_ADR_6',
'team1_p_asists_6',
'team1_p_death_6',
'team1_p_kast_6',
'team1_p_kills_6',
'team1_p_rating_6',
'team1_player_name_6',
'team2_p_ADR_6',
'team2_p_asists_6',
'team2_p_death_6',
'team2_p_kast_6',
'team2_p_kills_6',
'team2_p_rating_6',
'team2_player_name_6'])

In [71]:
df_data_new = df_data_new[['date', 'team1main', 'team2main', 'map', 'eventmain', 'link',
       'score_team1main', 'score_team2main', 'date_time', 'eventdetails',
       'map_number', 'match_type', 'score_team1details', 'score_team2details',
       'score_total', 'team1details', 'team1_clutches_won',
       'team1_first_kills', 'team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5',
       'team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5',
       'team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5', 'team1_player_name_1',
       'team1_player_name_2', 'team1_player_name_3', 'team1_player_name_4',
       'team1_player_name_5', 'team1_rating', 'team1_score_1half',
       'team1_score_2half', 'team1side', 'team2details', 'team2_clutches_won',
       'team2_first_kills', 'team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5', 'team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5', 'team2_p_death_1', 'team2_p_death_2',
       'team2_p_death_3', 'team2_p_death_4', 'team2_p_death_5',
       'team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5', 'team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5',
       'team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5', 'team2_player_name_1',
       'team2_player_name_2', 'team2_player_name_3', 'team2_player_name_4',
       'team2_player_name_5', 'team2_rating', 'team2_score_1half',
       'team2_score_2half', 'map_name']]

In [72]:
df_data=pd.concat([df_data, df_data_new])

In [73]:
df_data.to_pickle('df_data_connected.pkl')

In [61]:
df_data = df_data.drop_duplicates(['link'])
df_data = df_data[~df_data.team2_score_2half.isna()]

In [62]:
df_data = df_data.dropna(subset=['team2_score_2half'])

In [63]:
df_data.score_total[df_data.score_total.isna()] = '-'

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
df_data = df_data.dropna()

C:\Users\Marcin\Anaconda3\lib\site-packages\pandas\core\frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [66]:

lista_kolumn = ['team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5']

for kolumna in lista_kolumn:
    df_data[kolumna] = df_data[kolumna].apply(lambda x: get_kills(x))


In [67]:
df_data[['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]]=df_data[['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]].replace('-', np.nan)

In [68]:
df_data[['team1main','team2main','map','eventmain','link','match_type','score_total','team1_player_name_1',
       'team1_player_name_2', 'team1_player_name_3', 'team1_player_name_4',
       'team1_player_name_5','team1side','team2_player_name_1',
       'team2_player_name_2', 'team2_player_name_3', 'team2_player_name_4',
       'team2_player_name_5']].astype(str)
df_data[['score_team1main' , 'score_team2main','map_number','team1_clutches_won', 'team1_first_kills','team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5','team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5','team1_score_1half',
       'team1_score_2half', 'team2_clutches_won', 'team2_first_kills','team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_death_1', 'team2_p_death_2',
       'team2_p_death_3', 'team2_p_death_4', 'team2_p_death_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5','team2_score_1half',
       'team2_score_2half']].astype(int)
df_data[['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]].astype(float)


#powtorzone kolumny
df_data.drop(columns =['eventdetails', 'score_team1details','score_team2details','team1details','team2details','map_name'])

#nie bedziemy mieli ich w przyszlosci
df_data.drop(columns =['score_total'])

ValueError: could not convert string to float: '70.4%'

In [21]:
df_data = df_data.drop_duplicates(['link'])
df_data = df_data[~df_data.team2_score_2half.isna()]['link']

In [12]:
lista_linkow = df_data[df_data.team2_score_2half.isna()]['link']

406       https://www.hltv.org/stats/matches/mapstatsid/...
418       https://www.hltv.org/stats/matches/mapstatsid/...
435       https://www.hltv.org/stats/matches/mapstatsid/...
1019      https://www.hltv.org/stats/matches/mapstatsid/...
1180      https://www.hltv.org/stats/matches/mapstatsid/...
2299      https://www.hltv.org/stats/matches/mapstatsid/...
2304      https://www.hltv.org/stats/matches/mapstatsid/...
2310      https://www.hltv.org/stats/matches/mapstatsid/...
2317      https://www.hltv.org/stats/matches/mapstatsid/...
2369      https://www.hltv.org/stats/matches/mapstatsid/...
2407      https://www.hltv.org/stats/matches/mapstatsid/...
2422      https://www.hltv.org/stats/matches/mapstatsid/...
2931      https://www.hltv.org/stats/matches/mapstatsid/...
2933      https://www.hltv.org/stats/matches/mapstatsid/...
2935      https://www.hltv.org/stats/matches/mapstatsid/...
2941      https://www.hltv.org/stats/matches/mapstatsid/...
2964      https://www.hltv.org/stats/mat

In [19]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')


df = pd.DataFrame()
number_of_rows = len(lista_linkow)

for index, row in enumerate(lista_linkow):
    try:
        dict_temp={}
        link = row
        wd.get(link)
        dict_temp = get_info_from_stats()
        dict_temp.update(get_match_info())
        dict_temp.update(get_match_info_details())
        dict_temp.update(get_info_players())
        dict_temp.update({'link' : link})
        df = df.append(dict_temp,ignore_index=True)
        print(f'{index}/{number_of_rows}')
        clear_output(wait=True)
    except NoSuchElementException:
        pass
   

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=75.0.3770.100)


In [10]:
df_data[df_data['score_team1main'].isna()]

,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,eventdetails,...,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,map_name
9423,2015-04-03,Playing Ducks,Space Soldiers,mrg,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-03 12:00,Copenhagen Games 2015,...,0.29,DESPE,TuGuX,ONLY,e1,XANTARES,0.48,3,0,Mirage
9425,2015-04-03,Playing Ducks,Space Soldiers,cch,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-03 12:00,Copenhagen Games 2015,...,0.29,DESPE,TuGuX,ONLY,e1,XANTARES,0.48,3,0,Cache
9427,2015-04-03,Property,Lemondogs,cch,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-03 12:00,Copenhagen Games 2015,...,0.38,Griefmaster,tease,ozoo,darti,atter,0.57,3,0,Cache
9429,2015-04-03,Property,Lemondogs,inf,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-03 12:00,Copenhagen Games 2015,...,0.38,Griefmaster,tease,ozoo,darti,atter,0.57,3,0,Inferno
9439,2015-04-02,jQntefabrikkeN,Free Transfer,cch,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-02 23:30,Copenhagen Games 2015,...,0.43,Kroxie,Jialocin,hyaluronic,tmL,mithR,0.60,4,0,Cache
9441,2015-04-02,no_rhythm,Lions,mrg,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,19,2015-04-02 23:15,Copenhagen Games 2015,...,0.71,zanoj,xone,wezx,zekon,mAt,0.98,12,3,Mirage
9443,2015-04-02,Frederiksberg,schlaegeros,mrg,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-02 23:10,Copenhagen Games 2015,...,0.64,Lukki,AnageR,FONZEN,ComPooTer,Skytte,0.79,5,2,Mirage
9444,2015-04-02,Temp.no,myRevenge,inf,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-02 22:45,Copenhagen Games 2015,...,0.26,f0lM3r,tw1ce,krynex,biLi,gade,0.44,2,0,Inferno
9445,2015-04-02,TAKTIKSTATIV,malefiQ,d2,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,16,2015-04-02 22:30,Copenhagen Games 2015,...,0.47,muffey,Ryxxo,CeXo,mhz,da1men,0.56,3,0,Dust2
9447,2015-04-02,no_rhythm,partyastronauts,cch,Copenhagen Games 2015,https://www.hltv.org/stats/matches/mapstatsid/...,NaN,NaN,2015-04-02 21:40,Copenhagen Games 2015,...,0.64,firesenz,stb,QuESt,slickb,Cy,0.90,4,6,Cache


In [31]:

lista_kolumn = ['team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5']

for kolumna in lista_kolumn:
    df_data[kolumna] = df_data[kolumna].apply(lambda x: get_kills(x))



,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,eventdetails,...,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,map_name
0,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,...,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Train_se
2,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,...,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Inferno_se
4,2012-10-11,Red Dead,mousesports,Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,2,16,2012-10-11 20:00,ESL Pro Series Germany Winter Season 2012,...,0.28,fl0w,cavy,stefan,kev1n,caLipo,0.60,2,0,Inferno_se
5,2012-10-10,mousesports,Tt Dragons,Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-10 21:00,ESWC 2012 German Qualifier,...,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,Mirage_ce
7,2012-10-10,Tt Dragons,mousesports,Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16,2012-10-10 21:00,ESWC 2012 German Qualifier,...,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7,Dust2_se
9,2012-10-10,mousesports,ALTERNATE aTTaX,Dust2_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,16,7,2012-10-10 20:00,ESL Pro Series Germany Winter Season 2012,...,0.69,mnL,Rak,fel1x,SolEk,kRYSTAL,0.90,7,0,Dust2_se
10,2012-10-10,mTw,gamed!de,Nuke_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,14,2012-10-10 20:00,ESWC 2012 German Qualifier,...,0.53,r0b,syrsoN,enkay J,dynrmight,yannick,0.87,7,7,Nuke_se
13,2012-10-10,gamed!de,mTw,Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16,2012-10-10 20:00,ESWC 2012 German Qualifier,...,0.53,r0b,syrsoN,enkay J,dynrmight,yannick,0.87,7,7,Mirage_ce
16,2012-10-10,gamed!de,mTw,Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,13,2012-10-10 20:00,ESWC 2012 German Qualifier,...,0.53,r0b,syrsoN,enkay J,dynrmight,yannick,0.87,7,7,Dust2_se
19,2012-10-09,FMESPORTS,aliquo,Mirage_ce,ESWC 2012 UK Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,13,2012-10-09 22:00,ESWC 2012 UK Qualifier,...,0.59,spk,KYESAR,Murdy,Blacky,RAMPAGE,0.90,8,5,Mirage_ce


In [13]:
# wez z liczbe kill z razem z Headshots
def get_kills(row_text):
    kills = row_text.split(" ")[0]
    return kills

In [14]:
x = df_data.iloc[1][['team1_p_kills_1','team1_p_kills_2']]

AttributeError: 'vectorize' object has no attribute 'values'

In [32]:
df_data[['score_team1main' , 'score_team2main','map_number','team1_clutches_won', 'team1_first_kills','team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5','team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5','team1_score_1half',
       'team1_score_2half', 'team2_clutches_won', 'team2_first_kills','team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_death_1', 'team2_p_death_2',
       'team2_p_death_3', 'team2_p_death_4', 'team2_p_death_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5','team2_score_1half',
       'team2_score_2half']].astype(int)

,score_team1main,score_team2main,map_number,team1_clutches_won,team1_first_kills,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,...,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_score_1half,team2_score_2half
0,16,3,2,1,14,0,0,0,0,0,...,16,18,18,11,8,6,7,7,3,0
2,16,8,2,1,14,0,0,0,0,0,...,16,18,18,11,8,6,7,7,3,0
4,2,16,1,3,13,0,0,0,0,0,...,17,17,16,18,11,7,7,4,2,0
5,16,8,2,0,12,0,0,0,0,0,...,18,13,14,23,21,20,14,16,9,7
7,7,16,2,0,12,0,0,0,0,0,...,18,13,14,23,21,20,14,16,9,7
9,16,7,1,2,10,0,0,0,0,0,...,16,18,16,16,16,16,14,10,7,0
10,16,14,3,2,21,0,0,0,0,0,...,20,26,26,21,22,20,20,13,7,7
13,7,16,3,2,21,0,0,0,0,0,...,20,26,26,21,22,20,20,13,7,7
16,16,13,3,2,21,0,0,0,0,0,...,20,26,26,21,22,20,20,13,7,7
19,16,13,2,3,15,0,0,0,0,0,...,22,24,19,29,25,17,15,10,8,5


In [63]:
df_data[['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]]=df_data[['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]].replace('-', np.nan)

In [60]:
df_data[['team2_p_ADR_2','team1_p_ADR_1']]

,team2_p_ADR_2,team1_p_ADR_1
0,-,-
2,-,-
4,-,-
5,-,-
7,-,-
9,-,-
10,-,-
13,-,-
16,-,-
19,-,-


In [23]:
lista_kolumn = ['team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5']






,team1_p_kast_1,team1_p_kast_2
0,NaN,NaN
2,NaN,NaN
4,NaN,NaN
5,NaN,NaN
7,NaN,NaN
9,NaN,NaN
10,NaN,NaN
13,NaN,NaN
16,NaN,NaN
19,NaN,NaN


In [33]:
def remove_last_elment_from_str(cell):
    if isinstance(cell , str):
        cell=cell[:-1]
    return cell

In [25]:
lista_kolumn = ['team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5']

In [35]:
for kolumna in lista_kolumn:
    df_data[kolumna] = df_data[kolumna].apply(lambda x: remove_last_elment_from_str(x))

In [34]:
remove_last_elment_from_str(df_data.iloc[2]['team1_p_kast_1'])

nan

# 1.sciagamy jeszce te eventy

In [2]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/events')

In [73]:
def get_data_events():
    df=pd.DataFrame()
    df_new =  get_info_from_main_tabels_event()
    df = pd.concat([df, df_new])
    while if_next_page_events():
        next_page()
        df_new =  get_info_from_main_tabels_event()
        df = pd.concat([df, df_new])
    return df




def if_next_page_events():
    element = wd.find_elements_by_class_name('pagination-next')
    next_page_exists=True
    if element[0].get_attribute('href') == None:
        next_page_exists=False
    return next_page_exists




def get_info_from_main_tabels_event():
    results_table = []
    tables = wd.find_elements_by_class_name('stats-table') 
    
    for table in tables:
        rows = table.find_elements_by_tag_name('tr')
    
    for row in rows:
        columns = row.find_elements_by_css_selector("td")
        temp = []
        for column in columns:
            temp.append(column.text)
        results_table.append(temp)
        df_out = pd.DataFrame(results_table)
             
    return df_out



In [90]:
type_of_event = ['Lan',"Majors", 'BigEvents']
df_events = pd.DataFrame() 

wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/events')
    
for event_type in type_of_event:
    wd.get(f'https://www.hltv.org/stats/events?matchType={event_type}')
    df_temp = get_data_events()
    df_temp['type_of_event'] = event_type
    df_events = pd.concat([df_events,df_temp])


In [104]:
df_events.to_pickle('events.pkl')

In [62]:
if_next_page_events('Majors')

True

In [56]:
element = wd.find_elements_by_class_name('pagination-next')

In [57]:
element[0].get_attribute('href') == None

False

In [81]:


wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/events')

for event_type in type_of_event:
    wd.get(f'https://www.hltv.org/stats/events?matchType={event_type}')

In [100]:
df_events = df_events.dropna()
df_events.columns = ['id', 'event_name', 'Maps','type_of_event']

In [102]:
df_events

,id,event_name,Maps,type_of_event
1,,JLF League,6,Lan
2,,Good Game League 2019,23,Lan
3,,BLAST Pro Series Los Angeles 2019,21,Lan
4,,ESEA Season 31 Global Challenge,22,Lan
5,,Musimundo Gaming Week 2019,8,Lan
6,,The Gaming Stadium Summer Masters 2019,6,Lan
7,,ESEA MDL Season 31 Europe Finals,2,Lan
8,,ESEA MDL Season 31 North America Finals,2,Lan
9,,Up Expo Game 2019,7,Lan
10,,Polish Esport League Spring 2019 Finals,15,Lan
